In [1]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, URIRef,Namespace, Literal, OWL,RDFS, RDF, XSD,RDFS

import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
import ifcopenshell.util.selector
import random as rd
import pandas as pd
import cv2
from pathlib import Path
import pye57
import laspy
import copy
import datetime
import ezdxf 
from PIL import Image
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from collections import Counter
import json
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.utils.cadutils as cadu
import geomapi.utils.imageutils as iu
import geomapi.tools as tl
import sys
import trimesh

GEOMAPI_GRAPH=Graph().parse('../geomapi/ontology/geomapi_ontology.ttl')
GEOMAPI_PREFIXES = {prefix: Namespace(namespace) for prefix, namespace in GEOMAPI_GRAPH.namespace_manager.namespaces()}
GEOMAPI_NAMESPACE = Namespace('https://w3id.org/geomapi#')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
d2=1/np.sin(np.radians(45))
a=np.arcsin(d2/2)
print(d2,np.degrees(a))

1.414213562373095 44.99999999999999


In [18]:
np.sqrt(1.737**2+1.422**2)

2.2448280557762104

In [40]:

b=1.57/np.sin(np.deg2rad(45))
print(b)
np.arctan(2/b)*180/np.pi

2.220315292925759


42.01167154624617

41.84940739053202

In [ ]:
1.579*np.arcsin(1.415/2)*180/np.pi

45.03187078392833

In [2]:
# %autoreload 2

In [3]:
# %load_ext autoreload

In [4]:

#PATH
os.getcwd()
path= Path(os.getcwd()).parent / "tests" / "testfiles"  

In [13]:
100-np.arccos(6/30)*200/np.pi

12.81884336979499

In [9]:
dz=np.sqrt(11.236**2+9.56**2)
a=np.sqrt(30**2+dz**2)
db=np.sqrt(a**2+9.5**2)
print(a,db)

33.43114260685686 34.75473055570997


In [6]:
np.arccos(np.sqrt(6**2+9.5**2)/30)*200/np.pi

75.56035098498752

In [5]:
resource=np.full((50,100,3),0)
resource[10,0]=[1,0,0]
resource[10,1]=[1,0,0]
resource[10,2]=[1,0,0]
node=PanoNode(depthMap=np.full((50,100),1),resource=resource )
pcd=node.get_pcd_from_depth_map()
print(np.asarray(pcd.points))

[[-2.09130147e-17 -2.44200237e-16  1.00000000e+00]
 [-2.47966097e-17 -2.44076956e-16  1.00000000e+00]
 [-2.86645668e-17 -2.43707611e-16  1.00000000e+00]
 ...
 [-1.31614625e-17  2.00381599e-16 -1.00000000e+00]
 [-1.70294197e-17  2.00012253e-16 -1.00000000e+00]
 [-2.09130147e-17  1.99888973e-16 -1.00000000e+00]]


In [8]:
rays=node.create_rays()
lineset=gmu.rays_to_lineset(rays)
lineset.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd,lineset])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [6]:
#paint pcd black, color first point red
# pcd.paint_uniform_color([0,0,0])
pcd.colors[1000]=[0,1,0]
pcd.colors[1001]=[0,1,0]
pcd.colors[1002]=[0,1,0]
width=100
height=50
points=np.array([[width/2 ,height/2], # front
                [width/2,0], # top 
                [ width/4,height/2],  # left
                [width*3/4,height/2 ],# right
                [0,height/2],# back
                [width/2,height]]) # bottom
rays=node.create_rays(points) # top left
print(rays)
lineset=gmu.rays_to_lineset(rays)
lineset.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd,lineset])

[[-2.09130147e-17  3.90767080e-17  1.14198372e-17  6.40621553e-02
  -9.97945393e-01 -1.01653579e-03]
 [-2.09130147e-17  3.90767080e-17  1.14198372e-17  7.84633898e-18
  -9.97945393e-01  6.40702200e-02]
 [-2.09130147e-17  3.90767080e-17  1.14198372e-17 -9.99360409e-01
  -3.20515776e-02  1.58578122e-02]
 [-2.09130147e-17  3.90767080e-17  1.14198372e-17  9.95253822e-01
   9.60230259e-02 -1.57926490e-02]
 [-2.09130147e-17  3.90767080e-17  1.14198372e-17 -6.12246325e-17
   1.00000000e+00  9.71510091e-19]
 [-2.09130147e-17  3.90767080e-17  1.14198372e-17 -2.03281568e-03
  -9.97945393e-01 -6.40379633e-02]]


In [13]:
points

array([[50., 25.],
       [50.,  0.],
       [25., 25.],
       [75., 25.],
       [ 0., 25.],
       [50., 50.]])

In [26]:
points=np.array([[width/2 ,height/2], # front
                [width/2,0], # top 
                [ width/4,height/2],  # left
                [width*3/4,height/2 ],# right
                [0,height/2],# back
                [width/2,height]]) # bottom
result_world=np.array([[0,1,0], # front
                [0,0,1], # top 
                [-1,0,0],  # left
                [1,0,0],# right
                [0,-1,0],# back
                [0,0,-1]]) # bottom
result_local=np.array([[0,0,1], # front
                [0,1,0], # top 
                [-1,0,0],  # left
                [1,0,0],# right
                [0,0,-1],# back
                [0,-1,0]]) # bottom

In [8]:
#PANO
panoPath = path / 'pano'/ "00000-pano.jpg"
pano = Image.open(panoPath)
depthPath = path / 'pano'/ "00000-pano_depth.png"
depthMap = np.asarray(Image.open(depthPath))
csvPath = path / 'pano'/ "pano-poses.csv"
csv = pd.read_csv(csvPath)
jsonPath = path / 'pano'/ "00000-info.json"
panoGraphPath=path / 'graphs'/'pano_graph.ttl'
# panoGraph=Graph().parse(panoGraphPath, format='turtle')
print(f'    loaded {pano}')


    loaded <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=8192x4096 at 0x20DD7CD77C0>


In [9]:
ball=o3d.geometry.TriangleMesh.create_sphere(radius=0.5) #this gets Obox with x,y,z and no rotation
ball.translate([0,0,-1])
box=o3d.geometry.TriangleMesh.create_box(1,1,1)
box.translate([0,0,1])

#base hull
node= PanoNode(path=panoPath,depthPath=depthPath,getResource=True)
node.__dict__


{'_depthPath': WindowsPath('c:/Users/u0094523/code/geomapi/tests/testfiles/pano/00000-pano_depth.png'),
 '_depthMap': array([[14.11 , 14.117, 14.123, ..., 14.093, 14.098, 14.104],
        [14.11 , 14.117, 14.123, ..., 14.092, 14.098, 14.104],
        [14.109, 14.116, 14.124, ..., 14.091, 14.097, 14.103],
        ...,
        [ 2.044,  2.044,  2.044, ...,  2.043,  2.043,  2.043],
        [ 2.044,  2.044,  2.044, ...,  2.043,  2.043,  2.043],
        [ 2.044,  2.044,  2.044, ...,  2.043,  2.043,  2.043]]),
 '_imageWidth': 8192,
 '_imageHeight': 4096,
 '_jsonPath': None,
 '_resource': array([[[238, 239, 241],
         [238, 239, 241],
         [238, 239, 241],
         ...,
         [238, 239, 241],
         [238, 239, 241],
         [238, 239, 241]],
 
        [[238, 239, 241],
         [238, 239, 241],
         [238, 239, 241],
         ...,
         [238, 239, 241],
         [238, 239, 241],
         [238, 239, 241]],
 
        [[238, 239, 241],
         [238, 239, 241],
         [238,

In [10]:
pcd=node.get_pcd_from_depth_map()
# pcd.paint_uniform_color([0.5, 0.5, 0.5])

In [11]:
pcd.get_center()

array([6.76121701, 0.36516179, 2.96351198])

In [12]:
rays=node.create_rays(,depths=20)
print(rays)
lineset=gmu.rays_to_lineset(rays)
lineset.paint_uniform_color([0.5,0.5,0.5])

[[ 6.38405617e+00  2.90079733e-01  1.74535209e+00 -1.12667506e-15
  -1.37977900e-31  1.84000000e+01]
 [ 6.38405617e+00  2.90079733e-01  1.74535209e+00 -1.57628588e-12
  -1.93039345e-28  1.84000000e+01]
 [ 6.38405617e+00  2.90079733e-01  1.74535209e+00 -1.12667506e-15
  -1.93039345e-28  1.84000000e+01]
 [ 6.38405617e+00  2.90079733e-01  1.74535209e+00 -1.12667506e-15
   9.37275611e-28  1.84000000e+01]
 [ 6.38405617e+00  2.90079733e-01  1.74535209e+00 -1.12667506e-15
  -2.80760681e-27  1.84000000e+01]
 [ 6.38405617e+00  2.90079733e-01  1.74535209e+00  7.65343617e-12
   9.37275611e-28  1.84000000e+01]]


LineSet with 6 lines.

In [15]:
pcd.get_center()

array([6.76121701, 0.36516179, 2.96351198])

In [13]:
frame=o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0, origin=[0,0,0])
box=o3d.geometry.TriangleMesh.create_box(10,10,3)
box.paint_uniform_color([0.5,0.5,0.5])
#visualize
o3d.visualization.draw_geometries([frame,pcd,lineset])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [ ]:

#PARKING
#POINTCLOUD
pcdPath=path / 'pcd'/"parking.pcd"
pcd=o3d.io.read_point_cloud(str(pcdPath))
print(f'    loaded {pcd}')

e57Path1=path / 'pcd'/"lidar.e57"
e57XmlPath=path / 'pcd'/"lidar.xml"
e571 = pye57.E57( str(e57Path1))
e571Data=e571.read_scan_raw(1) 
print(f'    loaded {e57Path1}')

e57Path2=path / 'pcd'/"parking.e57"
e572 = pye57.E57( str(e57Path2))
e572Data=e572.read_scan_raw(0) 
print(f'    loaded {e57Path2}')

lasPath=path / 'pcd'/"parking.las"
las=laspy.read(str(lasPath))
print(f'    loaded {lasPath}')

pcdGraphPath=path / 'graphs' /  'pcd_graph.ttl'
pcdGraph=Graph().parse(pcdGraphPath)
pcdSubject=next(s for s in pcdGraph.subjects(RDF.type) if 'parking' in s.toPython() )
print(f'    loaded {pcdGraphPath}')           

#MESH
meshPath=path / 'mesh'/ 'parking.obj'
mesh=o3d.io.read_triangle_mesh(str(meshPath))
print(f'    loaded {mesh}') 
    
meshGraphPath=path / 'graphs' /  'mesh_graph.ttl'
meshGraph=Graph().parse(str(meshGraphPath))
meshSubject= next(s for s in meshGraph.subjects(RDF.type) if 'parking' in s.toPython() )
print(f'    loaded {meshGraphPath}')           

# #IFC
ifcPath=path / 'ifc' / "parking.ifc"
ifcWallPath=path / 'ifc' / "Basic_Wall_168_WA_f2_Soilmix_600mm_956569_06v1k9ENv8DhGMCvKUuLQV.ply"
# bimNodes=tl.ifc_to_nodes_multiprocessing(str(ifcPath)) #! Note: this uses geomapi functionality
# print(f'loaded {len(bimNodes)} bimNodes from ifc file')

ifcGraphPath=path /  'graphs' / 'parking_ifc_graph.ttl'
ifcGraph=Graph().parse(str(ifcGraphPath))
ifcSubject=next(s for s in ifcGraph.subjects(RDF.type))
print(f'    loaded {ifcGraphPath}')      

ifc = ifcopenshell.open(str(ifcPath))   
ifcSlab=ifc.by_guid('2qZtnImXH6Tgdb58DjNlmF')
ifcWall=ifc.by_guid('06v1k9ENv8DhGMCvKUuLQV')
ifcBeam=ifc.by_guid('05Is7PfoXBjhBcbRTnzewz' )
ifcColumn=ifc.by_guid('23JN72MijBOfF91SkLzf3a')

slabMesh=gmu.ifc_to_mesh(ifcSlab)
wallMesh=gmu.ifc_to_mesh(ifcWall)
beamMesh=gmu.ifc_to_mesh(ifcBeam)
columnMesh=gmu.ifc_to_mesh(ifcColumn) 
bimMeshes= [slabMesh,
                wallMesh,
                beamMesh,
                columnMesh]
bimBoxes=[mesh.get_oriented_bounding_box() for mesh in [slabMesh,
                                                                wallMesh,
                                                                beamMesh,
                                                                columnMesh] if mesh]
for box in bimBoxes:
    box.color = [1, 0, 0]


#IMG
# csvPath=path / 'img' / 'parking.csv' #! we don't do anything with the csv
imgGraphPath=path /  'graphs' / 'img_graph.ttl'
imgGraph=Graph().parse(str(imgGraphPath))
print(f'    loaded {imgGraphPath}')    

imageXmpPath1 = path / 'img' / 'DJI_0085.xmp'
imagePath1=path / 'img' / "DJI_0085.JPG" 
image1=cv2.imread(str(imagePath1))
imageCartesianTransform1= np.array([[-8.13902571e-02,  6.83059476e-01 ,-7.25813597e-01,  5.18276221e+01],
                                        [ 9.96648497e-01,  4.97790854e-02, -6.49139139e-02 , 6.10007435e+01],
                                        [-8.20972697e-03, -7.28664391e-01, -6.84821733e-01,  1.50408221e+01],
                                        [ 0.00000000e+00 , 0.00000000e+00, 0.00000000e+00 , 1.00000000e+00]])
imageSubject1=next((s for s in imgGraph.subjects() if 'DJI_0085' in s.toPython()),None )
principalPointV=-0.00481084380622187
principalPointU=-0.00219347744418651
print(f'    loaded {imagePath1}')           

imageXmpPath2 = path / 'img' / 'IMG_8834.xmp'
imagePath2=path / 'img' / "IMG_8834.JPG" 
image2=o3d.io.read_image(str(imagePath2))
imageCartesianTransform2= np.array([[ 4.12555151e-01,  4.12058430e-02 ,-9.10000179e-01, 6.68850552e+01],
                                        [ 9.10841440e-01, -4.52553581e-03,  4.12731621e-01 , 4.52551195e+01],
                                        [ 1.28887160e-02 ,-9.99140430e-01 ,-3.93990225e-02 , 5.45377093e+00],
                                        [ 0.00000000e+00 , 0.00000000e+00  ,0.00000000e+00 , 1.00000000e+00]])
imageSubject2=next((s for s in imgGraph.subjects() if 'IMG_8834' in s.toPython()),None )
print(f'    loaded {imagePath2}')    

# #RESOURCES
resourceGraphPath=path /  'graphs' / 'resources_graph.ttl'
resourceGraph=Graph().parse(str(resourceGraphPath))
print(f'    loaded {resourceGraphPath}')

#SET
setGraphPath=path /  'graphs' / 'set_graph.ttl'
setGraph=Graph().parse(str(setGraphPath))
print(f'    loaded {setGraphPath}')
        
#RESOURCES temporary folder
resourcePath= path / "resources"
if not os.path.exists(resourcePath):
    os.mkdir(resourcePath)

In [ ]:
nodes=tl.graph_path_to_nodes(resourceGraphPath,getResource=True)
pcdNode=next(n for n in nodes if isinstance(n,PointCloudNode))
meshNode=next(n for n in nodes if isinstance(n,MeshNode))
imageNode=next(n for n in nodes if isinstance(n,ImageNode))
bimNode=next(n for n in nodes if isinstance(n,BIMNode))
#big hull
points=o3d.utility.Vector3dVector()
for node in nodes:
    points.extend(node.convexHull.vertices)
pcd= o3d.geometry.PointCloud()
pcd.points=points
big_hull, _ =pcd.compute_convex_hull()
#big box
big_box =big_hull.get_oriented_bounding_box()
#big center
big_center = gmu.get_cartesian_transform(translation=big_hull.get_center())




In [12]:

# # #ROAD
# #CAD
# dxfPath= path / 'cad' / "road.dxf"
# cadPath= path / 'cad' / "line.ply"                
# dxf=ezdxf.readfile(dxfPath)
# entity = next(entity for entity in dxf.modelspace().query("INSERT"))
# insert=cadu.ezdxf_entity_to_o3d(entity)
# line=o3d.io.read_line_set(str(cadPath))
# cadGraphPath=path /  'graphs' / 'cad_graph.ttl'
# cadGraph=Graph().parse(cadGraphPath)
# print(f'loaded {dxf}')

# pcdPath=path / 'pcd'/"road.pcd"
# pcd=o3d.io.read_point_cloud(str(pcdPath))
# ifcPath=path / 'ifc' / "road.ifc"
# # bimNodes=tl.ifc_to_nodes_multiprocessing(str(ifcPath)) #! Note: this uses geomapi functionality
# # print(f'    loaded {len(bimNodes)} bimNodes from ifc file')

# meshPath=path / 'mesh'/"road.ply"
# mesh2=o3d.io.read_triangle_mesh(str(meshPath))
# print(f'    loaded {mesh2}')    

# #IMG
# imgGraphPath=path /  'graphs' / 'img_graph.ttl'
# imgGraph=Graph().parse(str(imgGraphPath))
# print(f'    loaded {imgGraphPath}')    


# imageXmlPath = path / 'img' / 'road.xml'            
# imagePath1=path / 'img' / "101_0367_0007.JPG" 
# image1=cv2.imread(str(imagePath1))
# imageCartesianTransform1= np.array([[-7.99965974e-01, -5.98493762e-01 ,-4.31237396e-02,  1.00592066e+05],
#                                         [-5.99164887e-01,  8.00618459e-01,  3.39417250e-03 , 1.96282855e+05],
#                                         [ 3.24942709e-02 , 2.85534531e-02, -9.99063973e-01,  3.19272496e+01],
#                                         [ 0.00000000e+00 , 0.00000000e+00,  0.00000000e+00 , 1.00000000e+00]])
# imageSubject1=next((s for s in imgGraph.subjects(RDF.type) if '101_0367_0007' in s.toPython()),None )
# focalLength1=3693.1569475809993
# imageWidth1=5472
# imageHeight1=3648
# print(f'    loaded {imagePath1}')           


# imagePath2=path / 'img' / "101_0367_0055.JPG" 
# image2=Image.open(imagePath2)
# imageCartesianTransform2= np.array([[ 8.16701918e-01,  5.76783553e-01,  1.78524640e-02,  1.00585779e+05],
#                                         [ 5.76947600e-01, -8.16762274e-01, -5.55470424e-03,  1.96265377e+05],
#                                         [ 1.13773570e-02 , 1.48364739e-02, -9.99825202e-01,  3.19327009e+01],
#                                         [ 0.00000000e+00, 0.00000000e+00 , 0.00000000e+00 , 1.00000000e+00]])
# imageSubject2=next((s for s in imgGraph.subjects(RDF.type) if '101_0367_0055' in s.toPython()),None )
# focalLength2=3693.1569475809993
# imageWidth2=5472
# imageHeight2=3648
# print(f'    loaded {imagePath2}')      

In [13]:
# #RAILWAY

# #CAD
# dxfPath= path / 'cad' / "railway.dxf"
# dxf=ezdxf.readfile(dxfPath)
# entity = next(entity for entity in dxf.modelspace().query("LINE"))
# line=cadu.ezdxf_entity_to_o3d(entity)
# cadGraphPath=path /  'graphs' / 'cad_graph.ttl'        
# cadGraph=Graph().parse(cadGraphPath)    
# cadSubject=next((s for s in cadGraph.subjects(RDF.type) if 'railway' in s.toPython()),None )
# #PCD
# pcdPath=path / 'pcd'/"railway.laz"
# laz=laspy.read(str(pcdPath))
# pcd=gmu.las_to_pcd(laz)
# print(f'    loaded {laz}')

# pcdGraphpath=path / 'graphs' /  'pcd_graph.ttl'
# pcdGraph=Graph().parse(str(pcdGraphpath))
# # pcdSubject=next(s for s in pcdGraph.subjects(RDF.type) if 'railway' in s.toPython() )
# print(f'    loaded {pcdGraphpath}')

# #MESH
# meshPath=path / 'mesh'/"railway.obj"
# mesh=o3d.io.read_triangle_mesh(str(meshPath))
# print(f'    loaded {mesh}')    
    
# meshGraphPath=path / 'graphs' /  'mesh_graph.ttl'
# meshGraph=Graph().parse(meshGraphPath)
# # meshSubject= next(s for s in meshGraph.subjects(RDF.type) if 'railway' in s.toPython() )
# print(f'    loaded {meshGraphPath}')    

# #IMG
# imgGraphPath=path /  'graphs' / 'img_graph.ttl'
# imgGraph=Graph().parse(str(imgGraphPath))
# print(f'    loaded {imgGraphPath}')    


# imageXmlPath = path / 'img' / 'railway.xml'            
# imagePath1=path / 'img' / "P0024688.JPG" 
# image1=cv2.imread(str(imagePath1))
# imageCartesianTransform1= np.array([[ 5.83812227e-02, -9.98234429e-01,  1.09387827e-02, 2.63374319e+05],
#                                     [-9.98294230e-01, -5.83833322e-02,  1.26659890e-04,1.51069035e+05],
#                                     [ 5.12206323e-04, -1.09275183e-02, -9.99940162e-01,2.82230717e+02],
#                                     [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,1.00000000e+00]])

# distance=25.98837076368911
# imageSubject1=next(s for s in imgGraph.subjects(RDF.type) if 'P0024688' in s.toPython() )
# focalLength1=21963.0445544689
# imageWidth1=11664
# imageHeight1=8750
# worldCoordinate= np.array([[263377.98, 151064.413 , 256.92,1]])
# imgCoordinate= np.array([[1676, 10007]]) 

# #ORTHO
# # orthoPath1=path / 'ortho' / "railway_0.01m.jpg"
# # orthoTfwPath1=path / 'ortho' / "railway_0.01m.tfw"
# # ortho1=Image.open(orthoPath1)
# # gsd1=0.01
# # orthoCenter1= np.array([151080.793, -263395.50100000005,256]) #how to we get height?
# # print(f'    loaded {orthoPath1}')  
# orthoPath2=path / 'ortho' / "railway-0-0.tif"
# orthoTfwPath2=path / 'ortho' / "railway-0-0.tfw"
# orthoDxfPath2=path / 'ortho' / "railway-scheme.dxf"        
# ortho2=Image.open(orthoPath2)
# gsd2=0.01560589037
# orthoCenter2= np.array([263379.5193, 151089.1667,256]) #how to we get height?
# orthoWidth2= 31.961
# orthoHeight2= 31.961
# orthoHeight=256
# print(f'    loaded {orthoPath2}')    
# orthoGraphPath=path /  'graphs' / 'ortho_graph.ttl'
# orthoGraph=Graph().parse(str(orthoGraphPath))
# orthoSubject=next(s for s in orthoGraph.subjects(RDF.type) if 'railway-0-0' in s.toPython() )

In [ ]:
graph=Graph()
graph=ut.bind_ontologies(graph)
string=ut.get_attribute_from_predicate(graph, predicate =Literal('http://libe57.org#pointCount')) 
print(string,'pointCount')
string=ut.get_attribute_from_predicate(graph, predicate =Literal('https://w3id.org/v4d/core#faceCount')) 
print(string,'faceCount')
string=ut.get_attribute_from_predicate(graph, predicate =Literal('http://purl.org/dc/terms/created')) 
print(string,'timestamp')
string=ut.get_attribute_from_predicate(graph, predicate =Literal('http://standards.buildingsmart.org/IFC/DEV/IFC2x3/TC1/OWL#className')) 
print(string,'className')
string=ut.get_attribute_from_predicate(graph, predicate =Literal('http://www.w3.org/2003/12/exif/ns#xResolution')) 
print(string,'xResolution')
string=ut.get_attribute_from_predicate(graph, predicate =Literal('https://w3id.org/gom#coordinateSystem')) 
print(string,'coordinateSystem')

In [ ]:
g=setNode.get_graph()
print(g.serialize())

In [31]:
lineset=gmu.mesh_get_lineset(big_hull)
lineset.paint_uniform_color([1,0,0])

nodelineset=gmu.mesh_get_lineset(node.convexHull)
nodelineset.paint_uniform_color([0,1,0])
# big_hull.paint_uniform_color([1,0,0])
#visualize the data
o3d.visualization.draw_geometries([pcdNode.resource,meshNode.resource,bimNode.resource,imageNode.convexHull,nodelineset,lineset])
# o3d.visualization.draw_geometries([node.convexHull,big_hull])


In [ ]:
lineSetNodes=tl.ezdxf_path_to_nodes(dxfPath)
lineSetNode=lineSetNodes[0].__dict__

In [ ]:
rays=imageNode.create_rays()
lineset1=gmu.rays_to_lineset(rays)

rays=imageNode.create_rays(np.array([1676,10007]),25)
lineset2=gmu.rays_to_lineset(rays)
lineset2.paint_uniform_color(np.array([  0, 189,   0])/255) 

In [ ]:
node.orientedBoundingBox.color=[1,0,0]

In [ ]:
#visualize mesh, rays and lineset 
# o3d.visualization.draw_geometries([lineset1,lineset2]+[n.resource for n in lineSetNodes])
geometries=gmu.join_geometries([n.resource for n in lineSetNodes])
o3d.visualization.draw_geometries([geometries,node.orientedBoundingBox,mesh])
# o3d.visualization.draw_geometries([lineset1,lineset2,geometries,imageNode.orientedBoundingBox, imageNode.convexHull])


In [ ]:
# tl.compute_graph_topology() #function that takes a graph and returns a graph with the topology of the graph

In [ ]:
translation=[1,2,3]
rotation=None
# Initialize identity rotation matrix and zero translation vector
r = np.eye(3)
t = np.zeros((3, 1))

# Update rotation matrix if provided
if rotation is not None:
    rotation=np.asarray(rotation)
    if rotation.size == 3:
        r = R.from_euler('xyz', rotation,degrees=True).as_matrix()
    elif rotation.size == 9:
        r = np.reshape(np.asarray(rotation), (3, 3))
    else:
        raise ValueError("Rotation must be either a 3x3 matrix or a tuple/list of three Euler angles.")

# Update translation vector if provided
if translation is not None:
    t = np.reshape(np.asarray(translation), (3, 1))

# Create the last row of the transformation matrix
h = np.array([[0, 0, 0, 1]])

# Concatenate rotation and translation to form the 3x4 upper part of the transformation matrix
transformation = np.hstack((r, t))

# Add the last row to make it a 4x4 matrix
transformation = np.vstack((transformation, h))
print(transformation)

In [ ]:
transformation = np.array([[1, 0, 0, 1],
                                   [0, 1, 0, 2],
                                   [0, 0, 1, 3],
                                   [0, 0, 0, 1]])
resource=copy.deepcopy(mesh)
center=resource.get_center()
node=Node(resource=resource,cartesianTransform=center)
print(node.cartesianTransform)
print(node.resource.get_center())


node.transform(transformation=transformation)

# node.transform(translation=translation)
print(node.cartesianTransform)
print(node.resource.get_center())
print(center[0]+1)
print(node.cartesianTransform[0,3])

In [ ]:
# imgNodes=[geomapi.imgNode(xmlPath=imageXmpPath1),imgNode(xmlPath=imageXmpPath1)]
imgNode1=ImageNode(xmpPath=imageXmpPath1,getResource=True)
imgNode2=ImageNode(xmpPath=imageXmpPath2,getResource=True)
meshNode=MeshNode(path=meshPath,getResource=True)
pcdNode=PointCloudNode(path=pcdPath,getResource=True)

In [ ]:
#get list of files
files=[str(path / 'IMG' / 'DJI_0085.xmp'),str(path / 'IMG' / 'IMG_8834.xmp')]
print(files)
#select every fifth files
imgNodes=[]
for f in files:     
    n=ImageNode(name=ut.get_filename(f),  #! this took quite long from remote drive                            
                                xmpPath=f,getResource=True)
    del n.resource
    if getattr(n,'imageWidth',None) is not None:
    #delete resource
    

        #set focallength and cartesianTransform
        n.focalLength35mm=n.focalLength35mm/36*n.imageWidth #! multiple definitions possible    
        rotation= n.cartesianTransform[0:3,0:3].T #! RC uses column-based rotaton matrix
        translation=n.cartesianTransform[0:3,3]
        n.cartesianTransform=gmu.get_cartesian_transform(rotation=rotation,translation=translation)
        
        imgNodes.append(n)
    
number=len(imgNodes)
imgNodes=[n for n in imgNodes if all(n.cartesianTransform[0:3,3] !=0)]
joinedImages=gmu.join_geometries([gmu.generate_visual_cone_from_image(n.cartesianTransform, height =1).paint_uniform_color([1,0,0]) for n in imgNodes])
print(f"{len(imgNodes)} / {number} Nodes created (remainder is not properly localised)!")

In [ ]:
tl.nodes_to_graph(imgNodes,graphPath=str(path / 'img' / 'parking_img_graph.ttl'),save=   True)

In [ ]:
ut.cartesianTransform_to_literal(imageCartesianTransform1)

In [ ]:
assert '-8.1390571e-02' in ut.cartesianTransform_to_literal(imageCartesianTransform1)